In [ ]:
!pip freeze | grep scikit-learn

In [4]:
import pickle
import pandas as pd
import numpy as np

In [5]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [6]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [7]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-03.parquet')
df.head(5)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,1,2022-03-01 00:13:08,2022-03-01 00:24:35,1.0,2.40,1.0,N,90,209,2,10.0,3.0,0.5,0.00,0.0,0.3,13.80,2.5,0.00,11.450000
1,1,2022-03-01 00:47:52,2022-03-01 01:00:08,1.0,2.20,1.0,N,148,234,2,10.5,3.0,0.5,0.00,0.0,0.3,14.30,2.5,0.00,12.266667
2,2,2022-03-01 00:02:46,2022-03-01 00:46:43,1.0,19.78,2.0,N,132,249,1,52.0,0.0,0.5,11.06,0.0,0.3,67.61,2.5,1.25,43.950000
3,2,2022-03-01 00:52:43,2022-03-01 01:03:40,2.0,2.94,1.0,N,211,66,1,11.0,0.5,0.5,4.44,0.0,0.3,19.24,2.5,0.00,10.950000
4,2,2022-03-01 00:15:35,2022-03-01 00:34:13,1.0,8.57,1.0,N,138,197,1,25.0,0.5,0.5,5.51,0.0,0.3,33.06,0.0,1.25,18.633333


In [8]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [9]:
np.std(y_pred)

5.556602554785001

Create a ride_id column

In [10]:
df['ride_id'] = f'{2022:04d}/{2:02d}_' + df.index.astype('str')

In [11]:
df_result = pd.concat([df['ride_id'], pd.DataFrame(y_pred, columns=["results"])], axis=1)
df_result.to_parquet(
    "model_output.parquet",
    engine='pyarrow',
    compression=None,
    index=False
)

In [12]:
df_result

,ride_id,results
0,2022/02_0,14.287470
1,2022/02_1,10.045199
2,2022/02_2,28.390979
3,2022/02_3,18.866162
4,2022/02_4,18.508053
...,...,...
3539757,NaN,10.556381
3539802,NaN,18.237859
3539803,NaN,11.053776
3539806,NaN,9.129053


In [15]:
df_result.results.mean()


12.758556818790895